In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/clicks-conversion-tracking/KAG_conversion_data.csv")
df.head()

In [ ]:
x=df.iloc[:, [6, 8]].values
x[0:5]

In [ ]:
print(df.xyz_campaign_id.unique())

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances


In [ ]:
def agn_Heirarchical(data, cutoff, linkage):
    #This is done using dynamic programming approach
    # if 1, it is single linkage else 2 is complete linkage, 3 is average linkage
    distance_matrix = euclidean_distances(data, data) #Step 1 - Calculate distance matrix
    distance_matrix = np.tril(distance_matrix) #Step 2 - Since matrix is symmetric, we just keep lower triangle matrix
    distance_matrix[distance_matrix == 0] = np.inf #Step 3 - Replace 0 by inf, it makes it easy for us to extract minimum using min function
    df = pd.DataFrame(data=np.ones(data.shape[0])*np.inf) #Initialized a dataframe which will store which point is in which cluster
    if cutoff > distance_matrix.shape[0]: #If user provides impractical cut-off, cluster everthing into one cluster and not listen to user 
        cutoff = distance_matrix.shape[0]
    if linkage == 1: #This 1 means formula of single linkage will be used, it is explained ahead
        d = {} #This dictionary keeps record of which data points or cluster are mer
        for i in range(0,cutoff):
            ij_min = np.unravel_index(distance_matrix.argmin(), distance_matrix.shape) #from the distance matrix, get the minimum distance
            #np.unravel_index gives us the position of minimum distance. e.g. (0,4) is where minimum value is present in matrix.
            #This is what we need as in Hierarchical clustering, we merge the two pairs with minimum distance
            if i == 0:
                df.iloc[ij_min[0]] = 0
                df.iloc[ij_min[1]] = 0
            else:
                try:
                    a = int(df.iloc[ij_min[0]])
                except:
                    df.iloc[ij_min[0]] = i
                    a = i
                try:
                    b = int(df.iloc[ij_min[1]])
                except:
                    df.iloc[ij_min[1]] = i
                    b = i
                df[(df[0]==a) | (df[0]==b)] = i
            d[i] = ij_min
            #The if, else code till here is just filling the dataframe as the two points/clusters comb
            for j in range(0, ij_min[0]):
                #we want to ignore the diagonal, and diagonal is 0. We replaced 0 by infinte. 
                #So this if condition will skip diagonals
                if np.isfinite(distance_matrix[ij_min[0]][j]) and np.isfinite(distance_matrix[ij_min[1]][j]):
                    #after two points/cluster are linked, to calculate new distance we take minimum distance for single linkage
                    distance_matrix[ij_min[1]][j] = min(distance_matrix[ij_min[0]][j], distance_matrix[ij_min[1]][j])
            # To avoid the combined data points/cluster in further calculations, we make them infinte.
            #Our if loop above this, will therefore skip the infinite record entries.
            distance_matrix[ij_min[0]] = np.inf
        return d, df[0].to_numpy()
    elif linkage == 2:
        d_complete = {}
        for i in range(0,cutoff):
            ij_min = np.unravel_index(distance_matrix.argmin(), distance_matrix.shape)
            if i == 0:
                df.iloc[ij_min[0]] = 0
                df.iloc[ij_min[1]] = 0
            else:
                try:
                    a = int(df.iloc[ij_min[0]])
                except:
                    df.iloc[ij_min[0]] = i
                    a = i
                try:
                    b = int(df.iloc[ij_min[1]])
                except:
                    df.iloc[ij_min[1]] = i
                    b = i
                df[(df[0]==a) | (df[0]==b)] = i
            d_complete[i] = ij_min
            for j in range(0, ij_min[0]):
                if np.isfinite(distance_matrix[ij_min[0]][j]) and np.isfinite(distance_matrix[ij_min[1]][j]):
#after two points/cluster are linked, to calculate new distance we take maximum distance for complete linkage
                    distance_matrix[ij_min[1]][j] = max(distance_matrix[ij_min[0]][j], distance_matrix[ij_min[1]][j])
            distance_matrix[ij_min[0]] = np.inf
        return d_complete, df[0].to_numpy()
    elif linkage == 3:
        d_average = {}
        for i in range(0,cutoff):
            ij_min = np.unravel_index(distance_matrix.argmin(), distance_matrix.shape)
            if i == 0:
                df.iloc[ij_min[0]] = 0
                df.iloc[ij_min[1]] = 0
            else:
                try:
                    a = int(df.iloc[ij_min[0]])
                except:
                    df.iloc[ij_min[0]] = i
                    a = i
                try:
                    b = int(df.iloc[ij_min[1]])
                except:
                    df.iloc[ij_min[1]] = i
                    b = i
                df[(df[0]==a) | (df[0]==b)] = i
            d_average[i] = ij_min
            for j in range(0, ij_min[0]):
                if np.isfinite(distance_matrix[ij_min[0]][j]) and np.isfinite(distance_matrix[ij_min[1]][j]):
    
#after two points/cluster are linked, to calculate new distance we take average distance for average linkage
                    distance_matrix[ij_min[1]][j] = (distance_matrix[ij_min[0]][j] + distance_matrix[ij_min[1]][j])/2.0          
            distance_matrix[ij_min[0]] = np.inf
        return d_average, df[0].to_numpy()





In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
iris_data = iris.data
d, target = agn_Heirarchical(iris_data, 147, 1)


In [ ]:
import scipy.cluster.hierarchy as shc
data = iris.data
plt.figure(figsize=(10, 7))
plt.title("Dendograms")
dend = shc.dendrogram(shc.linkage(data, method='ward'))



In [ ]:
target[target == 140] = 0
target[target == 144] = 1
target[target == 146] = 2


In [ ]:
plt.scatter(iris_data[:,1], iris_data[:,2], c=target, cmap="rainbow")


In [ ]:
plt.scatter(iris_data[:,2], iris_data[:,3], c=target, cmap="rainbow")


In [ ]:
plt.scatter(iris_data[:,1], iris_data[:,3], c=target, cmap="rainbow")


In [ ]:
d_complete, target_complete = agn_Heirarchical(iris_data, 147, 2)


In [ ]:
target_complete[target_complete == 141] = 0
target_complete[target_complete == 143] = 1
target_complete[target_complete == 146] = 2


In [ ]:
plt.scatter(iris_data[:,1], iris_data[:,3], c=target_complete, cmap="rainbow")


In [ ]:
plt.scatter(iris_data[:,1], iris_data[:,3], c=target_complete, cmap="rainbow")
            



In [ ]:
d_average, target_average = agn_Heirarchical(iris_data, 147, 3)

In [ ]:
target_average[target_average == 135] = 0
target_average[target_average == 143] = 1
target_average[target_average == 146] = 2


In [ ]:
plt.scatter(iris_data[:,1], iris_data[:,2], c=target_average, cmap="rainbow")


In [ ]:
plt.scatter(iris_data[:,2], iris_data[:,3], c=target_average, cmap="rainbow")


In [ ]:
plt.scatter(iris_data[:,1], iris_data[:,3], c=target_average, cmap="rainbow")


In [ ]:
print("Accuracy Average linkage :",100 - (iris.target != target_average).sum()/float(iris.target.size)*100)


In [ ]:
print("Accuracy single linkage : ",100 - (iris.target != target).sum()/float(iris.target.size)*100)


In [ ]:
print("Accuracy complete linkage : ",100 - (iris.target != target_complete).sum()/float(iris.target.size)*100)
